In [24]:
import pandas as pd
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = pd.read_csv('../data/basic_data.csv')

In [ ]:
data['log_closePrice'] = np.log(data['closePrice'])

In [ ]:
colint_stock_list = []

for stock_i in range(50):

    stock_i_df = data[data['stock'] == stock_i]

    for stock_j in range(stock_i+1, 50):
        
        stock_j_df = data[data['stock'] == stock_j]

        outcome = coint(stock_i_df['log_closePrice'], stock_j_df['log_closePrice'])

        if outcome[1] < 0.05:
            colint_stock_list.append(([stock_i, stock_j], outcome))
            print(stock_i, stock_j)


In [ ]:
for ([stock_i, stock_j], outcome) in colint_stock_list:
    stock_i_df = data[data['stock'] == stock_i]
    stock_j_df = data[data['stock'] == stock_j]

    print(stock_i, stock_j)

    plt.plot(stock_i_df['date'], stock_i_df['closePrice'], label=stock_i)
    plt.plot(stock_j_df['date'], stock_j_df['closePrice'], label=stock_j)
    plt.show()


In [ ]:
colint_stock_list_ct = []

for stock_i in range(50):

    stock_i_df = data[data['stock'] == stock_i]

    for stock_j in range(stock_i+1, 50):
        
        stock_j_df = data[data['stock'] == stock_j]

        outcome = coint(stock_i_df['log_closePrice'], stock_j_df['log_closePrice'], trend = 'ct')

        if outcome[1] < 0.05:
            colint_stock_list_ct.append(([stock_i, stock_j], outcome))
            print(stock_i, stock_j)


In [ ]:
len(colint_stock_list_ct)

In [ ]:
for ([stock_i, stock_j], outcome) in colint_stock_list_ct:
    stock_i_df = data[data['stock'] == stock_i]
    stock_j_df = data[data['stock'] == stock_j]

    print(stock_i, stock_j)

    plt.plot(stock_i_df['date'], stock_i_df['log_closePrice'], label=stock_i)
    plt.plot(stock_j_df['date'], stock_j_df['log_closePrice'], label=stock_j)
    plt.show()


In [26]:
colint_stock_list_ct[20]

([34, 36],
 (-3.835765014559812,
  0.04330999768121991,
  array([-4.35870257, -3.79967781, -3.51053148])))

In [51]:
from sklearn.linear_model import LinearRegression

coint_trend_diff = pd.DataFrame(columns=['Pair', 'Slope', 'intercept'])

for ([stock_i, stock_j], outcome) in colint_stock_list_ct:
    stock_i_df = data[data['stock'] == stock_i]
    stock_j_df = data[data['stock'] == stock_j]

    # Price difference df
    diff = pd.DataFrame()
    diff['difference'] = (stock_i_df['closePrice'].reset_index() - stock_j_df['closePrice'].reset_index())['closePrice']
    diff = np.array(diff['difference']).reshape(-1, 1)
    
    # Fit linear regression
    LR = LinearRegression(n_jobs=-1).fit(diff, stock_i_df['date'])

    # Append new data
    new_data = {
        'Pair': [(stock_i, stock_j)],
        'Slope': [LR.coef_],
        'intercept': [LR.intercept_]
    }
    new_df = pd.DataFrame(new_data)
    coint_trend_diff = coint_trend_diff.append(new_df, ignore_index=True)


coint_trend_diff

/var/folders/xk/rsnckpln5pl_8hrnss0pynlh0000gn/T/ipykernel_92085/4112147983.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coint_trend_diff = coint_trend_diff.append(new_df, ignore_index=True)
/var/folders/xk/rsnckpln5pl_8hrnss0pynlh0000gn/T/ipykernel_92085/4112147983.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coint_trend_diff = coint_trend_diff.append(new_df, ignore_index=True)
/var/folders/xk/rsnckpln5pl_8hrnss0pynlh0000gn/T/ipykernel_92085/4112147983.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coint_trend_diff = coint_trend_diff.append(new_df, ignore_index=True)
/var/folders/xk/rsnckpln5pl_8hrnss0pynlh0000gn/T/ipykernel_92085/4112147983.py:27: FutureWarning: The frame.append method is deprecated 

,Pair,Slope,intercept
0,"(0, 12)",[34.44346521012146],-1099.871659
1,"(0, 46)",[42.5741346700875],-239.365165
2,"(4, 35)",[-29.692380789612475],921.336435
3,"(10, 33)",[130.90305242508256],-158.317988
4,"(12, 27)",[43.66114385838506],1405.722187
5,"(13, 34)",[195.23601382845564],143.022183
6,"(16, 39)",[23.624626748127586],314.781931
7,"(18, 23)",[-53.163252586493705],1994.252027
8,"(18, 30)",[-60.19650645350908],1678.330297
9,"(18, 34)",[-60.643034733203194],2858.434913
